### Step 1: Install Required Libraries
```python
!pip install openai pymysql sqlalchemy pandas
```

In [ ]:
import openai
import pymysql
import pandas as pd
from sqlalchemy import create_engine


### Step 3: Configuration and OpenAI API Key
```python
# OpenAI API Key Configuration
openai.api_key = "your_openai_api_key_here"  # Replace with your OpenAI API Key

# MySQL Database Configuration
db_user = 'root'  # Replace with your MySQL username
db_password = 'password'  # Replace with your MySQL password
db_host = 'localhost'  # Host, e.g., localhost
db_name = 'genai_db'  # Database name

# MySQL Connection String
engine = create_engine(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')
connection = engine.connect()
```

### Step 4: Sample SQL Dataset Creation
```python
# Creating a sample table and inserting data for demonstration
query_create_table = '''
CREATE TABLE IF NOT EXISTS employees (
    id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(50),
    role VARCHAR(50),
    salary INT
);
'''

query_insert_data = '''
INSERT INTO employees (name, role, salary) VALUES
    ('Alice', 'Data Scientist', 120000),
    ('Bob', 'Software Engineer', 110000),
    ('Charlie', 'Product Manager', 115000);
'''

connection.execute(query_create_table)
connection.execute(query_insert_data)
```

In [ ]:
def get_mysql_data(sql_query):
    """
    Function to fetch data from MySQL database.
    """
    try:
        df = pd.read_sql(sql_query, connection)
        return df
    except Exception as e:
        return str(e)


In [ ]:
def query_chatgpt(prompt):
    """
    Sends a query to ChatGPT to generate an answer.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that responds to database queries."},
            {"role": "user", "content": prompt}
        ]
    )
    return response["choices"][0]["message"]["content"]


In [ ]:
def natural_language_to_sql(nl_query):
    """
    Function that converts a natural language query to SQL and fetches data.
    """
    chatgpt_prompt = f"""
    Convert the following natural language query to an SQL query for a MySQL database:
    "{nl_query}"

    Example Database Table: employees (id, name, role, salary)
    """

    sql_query = query_chatgpt(chatgpt_prompt)
    print("Generated SQL Query:\n", sql_query)
    result_df = get_mysql_data(sql_query)
    return result_df


In [ ]:
# Example Natural Language Queries
nl_query_1 = "Show me all employees with a salary greater than 110000."
nl_query_2 = "Who are the Data Scientists in the company?"

# Get Responses
response_df1 = natural_language_to_sql(nl_query_1)
print("\nResponse 1:\n", response_df1)

response_df2 = natural_language_to_sql(nl_query_2)
print("\nResponse 2:\n", response_df2)
